<a href="https://colab.research.google.com/github/quirogaez/capstone/blob/main/notebooks/05_TRAINING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 01 – Imports y configuración global
import os
import random
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.applications import DenseNet121, EfficientNetB4
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preproc
from tensorflow.keras.applications.efficientnet import preprocess_input as effnet_preproc

# reproducibilidad
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# mixed precision (si GPU lo soporta)
tf.config.optimizer.set_experimental_options({'auto_mixed_precision': True})
